## Uploads data sets

In [1]:
# libs
import pandas as pd

In [2]:
# Vendas
df_vendas = pd.read_parquet('fato_vendas_varejo.parquet')

# Filias

df_filiais = pd.read_parquet('filiais.parquet')

# Metas Vendas

df_metas_vendas = pd.read_parquet('metas_vendas_varejo.parquet')


In [3]:
# garantir tipos iguais
df_filiais["codigo_filial"] = pd.to_numeric(df_filiais["codigo_filial"], errors="coerce").astype("Int64")
df_metas_vendas["codigo_filial"] = pd.to_numeric(df_metas_vendas["codigo_filial"], errors="coerce").astype("Int64")
df_vendas["codigo_filial"] = pd.to_numeric(df_vendas["codigo_filial"], errors="coerce").astype("Int64")

fil_filiais = set(df_filiais["codigo_filial"].dropna().unique())
fil_metas   = set(df_metas_vendas["codigo_filial"].dropna().unique())
fil_vendas  = set(df_vendas["codigo_filial"].dropna().unique())

print("Qtd filiais:")
print("df_filiais:", len(fil_filiais))
print("df_metas_vendas:", len(fil_metas))
print("df_vendas:", len(fil_vendas))

print("\nFiliais em VENDAS e não em FILIAIS (qtd):", len(fil_vendas - fil_filiais))
print("Exemplos:", sorted(list(fil_vendas - fil_filiais))[:30])

print("\nFiliais em VENDAS e não em METAS (qtd):", len(fil_vendas - fil_metas))
print("Exemplos:", sorted(list(fil_vendas - fil_metas))[:30])

print("\nFiliais em METAS e não em FILIAIS (qtd):", len(fil_metas - fil_filiais))
print("Exemplos:", sorted(list(fil_metas - fil_filiais))[:30])

print("\nInterseção (filiais presentes nos 3):", len(fil_vendas & fil_metas & fil_filiais))


Qtd filiais:
df_filiais: 124
df_metas_vendas: 125
df_vendas: 125

Filiais em VENDAS e não em FILIAIS (qtd): 1
Exemplos: [np.int64(1704)]

Filiais em VENDAS e não em METAS (qtd): 0
Exemplos: []

Filiais em METAS e não em FILIAIS (qtd): 1
Exemplos: [np.int64(1704)]

Interseção (filiais presentes nos 3): 124


In [4]:
# foi identificado que a filial 1704 não existe no data set de vendas e metas de vendas
# entao optei pela remoção

filial_remover = 1704

df_vendas = df_vendas[df_vendas["codigo_filial"] != filial_remover].copy()
df_metas_vendas = df_metas_vendas[df_metas_vendas["codigo_filial"] != filial_remover].copy()

print("df_vendas filiais:", df_vendas["codigo_filial"].nunique())
print("df_metas_vendas filiais:", df_metas_vendas["codigo_filial"].nunique())
print("df_filiais filiais:", df_filiais["codigo_filial"].nunique())

df_vendas filiais: 124
df_metas_vendas filiais: 124
df_filiais filiais: 124


### Metas Vendas

In [ ]:
df_metas_vendas.head(10)

,codigo_filial,data_meta_venda,meta_n_med
0,1740,2025-01-01 03:00:00.000,4652.32
1,1830,2025-01-01 03:00:00.000,7870.85
2,1524,2025-01-01 03:00:00.000,6144.89
3,1677,2025-01-01 03:00:00.000,11840.37
4,1758,2025-01-01 03:00:00.000,9554.39
5,1656,2025-01-01 03:00:00.000,0.00
6,1581,2025-01-01 03:00:00.000,1680.03
7,1812,2025-01-01 03:00:00.000,8568.37
8,1746,2025-01-01 03:00:00.000,9320.65
9,1635,2025-01-01 03:00:00.000,7079.00


In [129]:
print("\n" + "="*60)
print("METAS shape:", df_metas_vendas.shape)

print("\n[METAS] missing por coluna (qtd):")
na_metas = df_metas_vendas.isna().sum().sort_values(ascending=False)
print(na_metas[na_metas > 0])

print("\n[METAS] duplicadas (linha inteira):", df_metas_vendas.duplicated().sum())

print("\n[METAS] duplicadas por (codigo_filial, data_meta_venda):",
      df_metas_vendas.duplicated(subset=["codigo_filial", "data_meta_venda"]).sum())


METAS shape: (42653, 3)

[METAS] missing por coluna (qtd):
Series([], dtype: int64)

[METAS] duplicadas (linha inteira): 0

[METAS] duplicadas por (codigo_filial, data_meta_venda): 0


In [5]:
df_metas_vendas["data"] = pd.to_datetime(df_metas_vendas["data_meta_venda"]).dt.normalize()

df_metas_vendas = (
    df_metas_vendas
    .sort_values(["codigo_filial", "data"])
    .reset_index(drop=True)
)

df_metas_vendas.head(10)

,codigo_filial,data_meta_venda,meta_n_med,data
0,1500,2025-01-01 03:00:00.000,0.00,2025-01-01
1,1500,2025-01-02 03:00:00.000,12732.76,2025-01-02
2,1500,2025-01-03 03:00:00.000,12732.76,2025-01-03
3,1500,2025-01-04 03:00:00.000,12732.76,2025-01-04
4,1500,2025-01-05 03:00:00.000,0.00,2025-01-05
5,1500,2025-01-06 03:00:00.000,22143.91,2025-01-06
6,1500,2025-01-07 03:00:00.000,22143.91,2025-01-07
7,1500,2025-01-08 03:00:00.000,24911.91,2025-01-08
8,1500,2025-01-09 03:00:00.000,22143.91,2025-01-09
9,1500,2025-01-10 03:00:00.000,19375.94,2025-01-10


In [136]:
df_metas_vendas.shape

(42653, 4)

### Informações Filiais

In [121]:
df_filiais.head(10)

,codigo_filial,faixa_vida,localidade,uf,tipo_estabelecimento,delivery,metragem_area_venda,panvel_clinic,estacionamento,atendimento_24_horas
0,1761,MAIS DE 3 ANOS,APUCARANA,PR,BAIRRO,SIM,503.5618,SIM,SIM,NÃO
1,1740,MAIS DE 3 ANOS,LONDRINA,PR,CENTRO,NÃO,531.0054,SIM,SIM,NÃO
2,1689,MAIS DE 3 ANOS,LONDRINA,PR,BAIRRO,NÃO,521.7424,NÃO,SIM,NÃO
3,1611,MAIS DE 3 ANOS,LONDRINA,PR,BAIRRO,NÃO,565.2000,SIM,SIM,NÃO
4,1764,MAIS DE 3 ANOS,LONDRINA,PR,CENTRO,NÃO,572.1708,SIM,SIM,NÃO
5,1884,MENOS DE 1 ANO,ARAPONGAS,PR,MALL,NÃO,471.2826,SIM,SIM,NÃO
6,1869,MENOS DE 1 ANO,TELEMACO BORBA,PR,CENTRO,SIM,566.5188,SIM,SIM,NÃO
7,1803,ENTRE 2-3 ANOS,CAMBE,PR,CENTRO,SIM,531.0682,SIM,SIM,NÃO
8,1782,MAIS DE 3 ANOS,ARAPONGAS,PR,CENTRO,SIM,585.9240,SIM,SIM,NÃO
9,1767,MAIS DE 3 ANOS,LONDRINA,PR,BAIRRO,SIM,626.6812,SIM,SIM,NÃO


In [87]:
print("\n" + "="*60)
print("FILIAIS shape:", df_filiais.shape)

print("\n[FILIAIS] missing por coluna (qtd):")
na_filiais = df_filiais.isna().sum().sort_values(ascending=False)
print(na_filiais[na_filiais > 0])

print("\n[FILIAIS] duplicadas (linha inteira):", df_filiais.duplicated().sum())

print("\n[FILIAIS] duplicadas por codigo_filial:", 
      df_filiais.duplicated(subset=["codigo_filial"]).sum())


FILIAIS shape: (124, 10)

[FILIAIS] missing por coluna (qtd):
Series([], dtype: int64)

[FILIAIS] duplicadas (linha inteira): 0

[FILIAIS] duplicadas por codigo_filial: 0


In [6]:
df_metas_vendas["data"] = pd.to_datetime(df_metas_vendas["data_meta_venda"]).dt.normalize()

df_metas_vendas = (
    df_metas_vendas
    .sort_values(["codigo_filial", "data"])
    .reset_index(drop=True)
)

In [ ]:
df_metas_vendas.head(31)

### Vendas

In [ ]:
df_vendas.tail()

In [8]:
print("VENDAS shape:", df_vendas.shape)

print("\n[VENDAS] missing por coluna (qtd):")
na_vendas = df_vendas.isna().sum().sort_values(ascending=False)
print(na_vendas[na_vendas > 0])

print("\n[VENDAS] duplicadas (linha inteira):", df_vendas.duplicated().sum())

print("\n[VENDAS] duplicadas por codigo_documento_saida:", 
      df_vendas.duplicated(subset=["codigo_documento_saida"]).sum())

print("\n[VENDAS] quantas linhas por (codigo_filial, data_emissao) — ajuda a ver se é 1/dia ou várias vendas/dia:")
print(df_vendas.groupby(["codigo_filial", "data_emissao"]).size().describe())



VENDAS shape: (5510993, 5)

[VENDAS] missing por coluna (qtd):
Series([], dtype: int64)

[VENDAS] duplicadas (linha inteira): 0

[VENDAS] duplicadas por codigo_documento_saida: 3755165

[VENDAS] quantas linhas por (codigo_filial, data_emissao) — ajuda a ver se é 1/dia ou várias vendas/dia:
count    42191.000000
mean       130.620109
std         67.159844
min          1.000000
25%         91.000000
50%        117.000000
75%        150.000000
max       1144.000000
dtype: float64


In [ ]:
#loja = 1500

# --- FILTROS ---
#v = df_vendas[df_vendas["codigo_filial"] == loja].copy()
#m = df_metas_vendas[df_metas_vendas["codigo_filial"] == loja].copy()

#v["data_emissao"] = pd.to_datetime(v["data_emissao"])
#v = v.sort_values(["data_emissao", "codigo_documento_saida"]).reset_index(drop=True)

#v.head(20)

#print("linhas:", len(v))
#print("documentos únicos:", v["codigo_documento_saida"].nunique())
#print("duplicados (por documento):", v.duplicated(subset=["codigo_documento_saida"]).sum())

# AQUI EU FIZ UM VERIFICAÇÃO PARA ENTENDER O ID DAS LOJAS E TAMBÉM O CODIGO DOCUMENTO SAIDA
# CADA LINHA É UMA VENDA
# ENTÃO DEFINIR AGRUPAR POR ID DE FILIAL E DATA DA VENDA SOMANDO QUANTIADE DE PRODUTOS VENDIDOS E FATURAMENTO
# FEITO ISSO COMPAREI COM AS METAS DIARIAS DE CADA FILIAL DO DATA SET META DE VENDAS VAREJO

linhas: 54810
documentos únicos: 54810
duplicados (por documento): 0


In [9]:
df_vendas["data"] = pd.to_datetime(df_vendas["data_emissao"]).dt.normalize()

vendas_dia = (
    df_vendas
    .groupby(["codigo_filial", "data"], as_index=False)
    .agg(
        quantidade=("quantidade", "sum"),
        faturamento=("faturamento", "sum"),
        n_vendas=("codigo_documento_saida", "nunique"),  # opcional: qtd de vendas no dia
    )
    .sort_values(["codigo_filial", "data"])
    .reset_index(drop=True)
)

In [10]:
print("dtype vendas_dia:", vendas_dia["codigo_filial"].dtype)
print("dtype df_filiais:", df_filiais["codigo_filial"].dtype)

dtype vendas_dia: Int64
dtype df_filiais: Int64


In [11]:
df_vendas_final = vendas_dia.merge(df_filiais, on="codigo_filial", how="left")

# checagem rápida: filiais sem match
print("Filiais sem match (linhas):", df_vendas_final["faixa_vida"].isna().sum())

Filiais sem match (linhas): 0


In [12]:
df_vendas_final

,codigo_filial,data,quantidade,faturamento,n_vendas,faixa_vida,localidade,uf,tipo_estabelecimento,delivery,metragem_area_venda,panvel_clinic,estacionamento,atendimento_24_horas
0,1500,2025-01-02,828.0,18077.83704,138,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
1,1500,2025-01-03,1107.0,29631.50298,188,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
2,1500,2025-01-04,597.0,14137.64100,98,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
3,1500,2025-01-06,975.0,21558.72465,166,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
4,1500,2025-01-07,969.0,25274.68470,165,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42186,1887,2025-12-27,597.0,17850.20877,99,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42187,1887,2025-12-28,291.0,9301.56894,62,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42188,1887,2025-12-29,537.0,17140.43700,88,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42189,1887,2025-12-30,735.0,16174.26540,106,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO


In [13]:
df_vendas_meta_final = df_metas_vendas.merge(df_filiais, on="codigo_filial", how="left")
print("Filiais sem match (linhas):", df_vendas_final["faixa_vida"].isna().sum())

Filiais sem match (linhas): 0


In [14]:
df_vendas_meta_final

,codigo_filial,data_meta_venda,meta_n_med,data,faixa_vida,localidade,uf,tipo_estabelecimento,delivery,metragem_area_venda,panvel_clinic,estacionamento,atendimento_24_horas
0,1500,2025-01-01 03:00:00.000,0.00,2025-01-01,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
1,1500,2025-01-02 03:00:00.000,12732.76,2025-01-02,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
2,1500,2025-01-03 03:00:00.000,12732.76,2025-01-03,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
3,1500,2025-01-04 03:00:00.000,12732.76,2025-01-04,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
4,1500,2025-01-05 03:00:00.000,0.00,2025-01-05,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
42648,1887,2025-12-27 03:00:00.000,15986.05,2025-12-27,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42649,1887,2025-12-28 03:00:00.000,16638.55,2025-12-28,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42650,1887,2025-12-29 03:00:00.000,16312.30,2025-12-29,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42651,1887,2025-12-30 03:00:00.000,14681.07,2025-12-30,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO


In [15]:
n_zero = (df_vendas_meta_final["meta_n_med"] == 0).sum()
print("meta_n_med == 0:", n_zero)

meta_n_med == 0: 627


In [16]:
filiais_meta_zero = (
    df_vendas_meta_final.loc[df_vendas_meta_final["meta_n_med"] == 0, "codigo_filial"]
    .dropna()
    .unique()
)

filiais_meta_zero

<IntegerArray>
[1500, 1509, 1530, 1545, 1548, 1578, 1590, 1617, 1629, 1647, 1656, 1695, 1698,
 1701, 1713, 1716, 1734, 1749, 1845, 1854, 1857, 1860, 1872, 1875, 1887]
Length: 25, dtype: Int64

In [17]:
meta_zero_por_filial = (
    df_vendas_meta_final[df_vendas_meta_final["meta_n_med"] == 0]
    .groupby("codigo_filial")
    .size()
    .sort_values(ascending=False)
)

meta_zero_por_filial

codigo_filial
1500    63
1509    62
1548    62
1749    62
1578    57
1590    54
1734    49
1872    31
1875    31
1860    31
1854    30
1857    21
1716    19
1617    16
1845    15
1530     8
1695     4
1887     4
1701     2
1629     1
1545     1
1713     1
1647     1
1656     1
1698     1
dtype: int64

## Analise descritiva para o df_vendas_final


In [19]:
df_vendas_final

,codigo_filial,data,quantidade,faturamento,n_vendas,faixa_vida,localidade,uf,tipo_estabelecimento,delivery,metragem_area_venda,panvel_clinic,estacionamento,atendimento_24_horas
0,1500,2025-01-02,828.0,18077.83704,138,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
1,1500,2025-01-03,1107.0,29631.50298,188,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
2,1500,2025-01-04,597.0,14137.64100,98,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
3,1500,2025-01-06,975.0,21558.72465,166,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
4,1500,2025-01-07,969.0,25274.68470,165,MAIS DE 3 ANOS,CURITIBA,PR,CENTRO,NÃO,309.0388,NÃO,NÃO,NÃO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42186,1887,2025-12-27,597.0,17850.20877,99,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42187,1887,2025-12-28,291.0,9301.56894,62,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42188,1887,2025-12-29,537.0,17140.43700,88,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO
42189,1887,2025-12-30,735.0,16174.26540,106,MENOS DE 1 ANO,MARINGA,PR,SUPERMERCADO,NÃO,632.4274,SIM,SIM,NÃO


In [20]:
top10_n_vendas = df_vendas_final.groupby('codigo_filial')['n_vendas'].sum().sort_values(ascending=False).head(10)
filiais_top_n_vendas = top10_n_vendas.index.tolist()
df_top10_n_vendas = df_vendas_final[df_vendas_final['codigo_filial'].isin(filiais_top_n_vendas)]

print("Top 10 filiais por número de vendas:")
print(top10_n_vendas)
print("\n")

Top 10 filiais por número de vendas:
codigo_filial
1698    134990
1713    127289
1557    120743
1521    120187
1545    119714
1626     86758
1614     85804
1647     79883
1629     73472
1689     72464
Name: n_vendas, dtype: int64




In [23]:
# Converter colunas numéricas
df_vendas_final['n_vendas'] = pd.to_numeric(df_vendas_final['n_vendas'])
df_vendas_final['faturamento'] = pd.to_numeric(df_vendas_final['faturamento'])

# 1. TOP 10 FILIAIS POR NÚMERO DE VENDAS (com todas características)
# Calcular total de vendas por filial
vendas_por_filial = df_vendas_final.groupby('codigo_filial')['n_vendas'].sum()

# Pegar as 10 filiais com maior número de vendas
top10_vendas = vendas_por_filial.nlargest(10).index.tolist()

# Filtrar o dataset original para essas filiais
df_top10_vendas = df_vendas_final[df_vendas_final['codigo_filial'].isin(top10_vendas)]

print("TOP 10 FILIAIS COM MAIOR NÚMERO DE VENDAS")
print("=" * 50)
print(f"Filiais: {top10_vendas}")
print(f"Total de registros: {len(df_top10_vendas)}")
print("\nPrimeiras linhas:")
print(df_top10_vendas.head())



TOP 10 FILIAIS COM MAIOR NÚMERO DE VENDAS
Filiais: [1698, 1713, 1557, 1521, 1545, 1626, 1614, 1647, 1629, 1689]
Total de registros: 3640

Primeiras linhas:
      codigo_filial       data  quantidade   faturamento  n_vendas  \
2442           1521 2025-01-01      1542.0   59153.54211       171   
2443           1521 2025-01-02      2910.0  136230.69816       324   
2444           1521 2025-01-03      2496.0  105922.65519       262   
2445           1521 2025-01-04      2148.0   90479.11203       241   
2446           1521 2025-01-05      2037.0   87497.95752       220   

          faixa_vida localidade  uf tipo_estabelecimento delivery  \
2442  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   
2443  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   
2444  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   
2445  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   
2446  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   

      me

In [26]:
# 2. TOP 10 FILIAIS POR FATURAMENTO (com todas características)
# Calcular total de faturamento por filial
faturamento_por_filial = df_vendas_final.groupby('codigo_filial')['faturamento'].sum()

# Pegar as 10 filiais com maior faturamento
top10_faturamento = faturamento_por_filial.nlargest(10).index.tolist()

# Filtrar o dataset original para essas filiais
df_top10_faturamento = df_vendas_final[df_vendas_final['codigo_filial'].isin(top10_faturamento)]

print("\n\nTOP 10 FILIAIS COM MAIOR FATURAMENTO")
print("=" * 50)
print(f"Filiais: {top10_faturamento}")
print(f"Total de registros: {len(df_top10_faturamento)}")
print("\nPrimeiras linhas:")
print(df_top10_faturamento.head())



TOP 10 FILIAIS COM MAIOR FATURAMENTO
Filiais: [1521, 1698, 1557, 1614, 1545, 1689, 1713, 1554, 1752, 1725]
Total de registros: 3644

Primeiras linhas:
      codigo_filial       data  quantidade   faturamento  n_vendas  \
2442           1521 2025-01-01      1542.0   59153.54211       171   
2443           1521 2025-01-02      2910.0  136230.69816       324   
2444           1521 2025-01-03      2496.0  105922.65519       262   
2445           1521 2025-01-04      2148.0   90479.11203       241   
2446           1521 2025-01-05      2037.0   87497.95752       220   

          faixa_vida localidade  uf tipo_estabelecimento delivery  \
2442  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   
2443  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   
2444  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   
2445  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   
2446  MAIS DE 3 ANOS   CURITIBA  PR               BAIRRO      SIM   

      metra

In [31]:
# Primeiro, garantir que temos uma linha única por filial com suas características
# (assumindo que as características são consistentes para cada filial)
#filiais_unicas = df_vendas_final.drop_duplicates('codigo_filial')

# Versão mais direta
print(f"Total filiais: {df_vendas_final['codigo_filial'].nunique()}")
print(f"Com estacionamento: {(df_vendas_final.drop_duplicates('codigo_filial')['estacionamento'] == 'SIM').sum()}")
print(f"Com atendimento 24h: {(df_vendas_final.drop_duplicates('codigo_filial')['atendimento_24_horas'] == 'SIM').sum()}")
print(f"Com Panvel Clinic: {(df_vendas_final.drop_duplicates('codigo_filial')['panvel_clinic'] == 'SIM').sum()}")
print(f"Com delivery: {(df_vendas_final.drop_duplicates('codigo_filial')['delivery'] == 'SIM').sum()}")

Total filiais: 124
Com estacionamento: 111
Com atendimento 24h: 4
Com Panvel Clinic: 87
Com delivery: 38


In [28]:
# Criar um resumo tabular
resumo = pd.DataFrame({
    'Característica': ['Estacionamento', 'Atendimento 24h', 'Panvel Clinic', 'Delivery'],
    'Total SIM': [
        contagem_estacionamento.get('SIM', 0),
        contagem_atendimento_24h.get('SIM', 0),
        contagem_panvel_clinic.get('SIM', 0),
        contagem_delivery.get('SIM', 0)
    ],
    'Total NÃO': [
        contagem_estacionamento.get('NÃO', 0),
        contagem_atendimento_24h.get('NÃO', 0),
        contagem_panvel_clinic.get('NÃO', 0),
        contagem_delivery.get('NÃO', 0)
    ],
    '% SIM': [
        f"{(contagem_estacionamento.get('SIM', 0)/len(filiais_unicas))*100:.1f}%",
        f"{(contagem_atendimento_24h.get('SIM', 0)/len(filiais_unicas))*100:.1f}%",
        f"{(contagem_panvel_clinic.get('SIM', 0)/len(filiais_unicas))*100:.1f}%",
        f"{(contagem_delivery.get('SIM', 0)/len(filiais_unicas))*100:.1f}%"
    ]
})

print("RESUMO DE CARACTERÍSTICAS DAS FILIAIS")
print("=" * 50)
print(resumo.to_string(index=False))

RESUMO DE CARACTERÍSTICAS DAS FILIAIS
 Característica  Total SIM  Total NÃO % SIM
 Estacionamento        111         13 89.5%
Atendimento 24h          4        120  3.2%
  Panvel Clinic         87         37 70.2%
       Delivery         38         86 30.6%


In [29]:
# Filiais COM estacionamento
filiais_com_estacionamento = filiais_unicas[filiais_unicas['estacionamento'] == 'SIM']['codigo_filial'].tolist()
print(f"Filiais COM estacionamento ({len(filiais_com_estacionamento)}): {filiais_com_estacionamento}")

# Filiais COM atendimento 24h
filiais_com_24h = filiais_unicas[filiais_unicas['atendimento_24_horas'] == 'SIM']['codigo_filial'].tolist()
print(f"\nFiliais COM atendimento 24h ({len(filiais_com_24h)}): {filiais_com_24h}")

# Filiais COM Panvel Clinic
filiais_com_clinic = filiais_unicas[filiais_unicas['panvel_clinic'] == 'SIM']['codigo_filial'].tolist()
print(f"\nFiliais COM Panvel Clinic ({len(filiais_com_clinic)}): {filiais_com_clinic}")

# Filiais COM delivery
filiais_com_delivery = filiais_unicas[filiais_unicas['delivery'] == 'SIM']['codigo_filial'].tolist()
print(f"\nFiliais COM delivery ({len(filiais_com_delivery)}): {filiais_com_delivery}")

Filiais COM estacionamento (111): [1503, 1506, 1512, 1515, 1518, 1521, 1524, 1527, 1530, 1536, 1539, 1542, 1545, 1551, 1554, 1557, 1560, 1563, 1566, 1569, 1572, 1575, 1581, 1587, 1590, 1593, 1596, 1599, 1602, 1605, 1608, 1611, 1614, 1617, 1623, 1626, 1629, 1635, 1638, 1641, 1644, 1647, 1650, 1653, 1656, 1659, 1665, 1668, 1671, 1674, 1677, 1683, 1686, 1689, 1692, 1698, 1701, 1707, 1710, 1713, 1716, 1719, 1725, 1728, 1731, 1734, 1737, 1740, 1743, 1746, 1749, 1752, 1755, 1758, 1761, 1764, 1767, 1770, 1773, 1782, 1785, 1788, 1791, 1794, 1797, 1800, 1803, 1806, 1809, 1812, 1815, 1824, 1830, 1833, 1836, 1842, 1845, 1848, 1851, 1854, 1857, 1860, 1863, 1866, 1869, 1872, 1875, 1878, 1881, 1884, 1887]

Filiais COM atendimento 24h (4): [1554, 1557, 1605, 1692]

Filiais COM Panvel Clinic (87): [1503, 1509, 1512, 1518, 1524, 1536, 1545, 1557, 1563, 1566, 1590, 1593, 1596, 1599, 1602, 1611, 1614, 1626, 1635, 1638, 1641, 1644, 1653, 1656, 1659, 1662, 1665, 1677, 1683, 1686, 1695, 1698, 1710, 1719, 17

In [35]:
filiais_unicas = df_vendas_final.drop_duplicates('codigo_filial')

print("Localidade:", filiais_unicas['localidade'].value_counts(), sep='\n')

Localidade:
localidade
CURITIBA                53
LONDRINA                14
MARINGA                 13
CASCAVEL                 4
SAO JOSE DOS PINHAIS     3
PONTA GROSSA             3
TOLEDO                   3
FOZ DO IGUACU            3
PINHAIS                  2
CIANORTE                 2
UMUARAMA                 2
PARANAVAI                2
CAMPO MOURAO             2
FRANCISCO BELTRAO        2
ARAPONGAS                2
GUARAPUAVA               2
PATO BRANCO              1
APUCARANA                1
CAMPO LARGO              1
CAMBE                    1
SARANDI                  1
PIRAQUARA                1
COLOMBO                  1
MEDIANEIRA               1
ARAUCARIA                1
TELEMACO BORBA           1
GOIOERE                  1
PARANAGUA                1
Name: count, dtype: int64


In [36]:
print("\nFaixa de vida:", filiais_unicas['faixa_vida'].value_counts(), sep='\n')


Faixa de vida:
faixa_vida
MAIS DE 3 ANOS    93
MENOS DE 1 ANO    12
ENTRE 1-2 ANOS    11
ENTRE 2-3 ANOS     8
Name: count, dtype: int64


In [37]:
print("\nUF:", filiais_unicas['uf'].value_counts(), sep='\n')
print("\nTipo estabelecimento:", filiais_unicas['tipo_estabelecimento'].value_counts(), sep='\n')


UF:
uf
PR    124
Name: count, dtype: int64

Tipo estabelecimento:
tipo_estabelecimento
BAIRRO          76
CENTRO          38
SHOPPING         6
MALL             3
SUPERMERCADO     1
Name: count, dtype: int64
